# Reconhecimento de texto manuscrito - testando o modelo

# Importando as bibliotecas

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import imutils
#from google.colab.patches import cv2_imshow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Carregando a rede neural

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
rede_neural = load_model('src/modelo/rede_neural')
rede_neural.summary()

# Carregando a imagem de teste

In [ ]:
img = cv2.imread('src/img/1.png')
plt.imshow(img)
plt.axis('off')  
plt.show()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray)
plt.axis('off')  
plt.show()

# Pré-processamento da imagem

In [ ]:
desfoque = cv2.GaussianBlur(gray, (3,3), 0)
plt.imshow(desfoque)
plt.axis('off')  
plt.show()

In [ ]:
adapt_media = cv2.adaptiveThreshold(desfoque, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
plt.imshow(adapt_media)
plt.axis('off')  
plt.show()

In [ ]:
inv = 255 - adapt_media
plt.imshow(inv)
plt.axis('off')  
plt.show()

In [ ]:
dilatado = cv2.dilate(inv, np.ones((3,3)))
plt.imshow(dilatado)
plt.axis('off')  
plt.show()

In [ ]:
bordas = cv2.Canny(desfoque, 40, 150)
plt.imshow(bordas)
plt.axis('off')  
plt.show()

In [ ]:
dilatado = cv2.dilate(bordas, np.ones((3,3)))
plt.imshow(dilatado)
plt.axis('off')  
plt.show()

# Detecção de contornos

In [ ]:
def encontrar_contornos(img):
  conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sort_contours(conts, method='left-to-right')[0]
  return conts

In [ ]:
conts = encontrar_contornos(dilatado.copy())

In [ ]:
conts

In [ ]:
l_min, l_max = 4, 160
a_min, a_max = 14, 140

In [ ]:
caracteres = []
img_cp = img.copy()
for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  #print(x, y, w, h)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    roi = gray[y:y+ h, x:x + w]
    #cv2_imshow(roi)
    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    plt.imshow(thresh)
    plt.axis('off')  
    plt.show()
    cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
plt.imshow(img_cp)
plt.axis('off')  
plt.show()

# Processando os caracteres detectados

## Extração ROI


In [ ]:
def extra_roi(img):
  roi = img[y:y + h, x:x + w]
  return roi

## Limiarização

In [ ]:
def limiarizacao(img):
  thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  return thresh

## Redimensionamento

In [ ]:
def redimensiona_img(img, l, a):
  if l > a:
    redimensionada = imutils.resize(img, width=28)
  else:
    redimensionada = imutils.resize(img, height=28)

  (a, l) = redimensionada.shape
  dX = int(max(0, 28 - l) / 2.0)
  dY = int(max(0, 28 - a) / 2.0)

  preenchida = cv2.copyMakeBorder(redimensionada, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))
  preenchida = cv2.resize(preenchida, (28, 28))
  return preenchida

In [ ]:
(x, y, w, h) = cv2.boundingRect(conts[6])
print(x, y, w, h)
img_teste = limiarizacao(gray[y:y+h, x:x+w])
plt.imshow(img_teste)
plt.axis('off')  
plt.show()
(a, l) = img_teste.shape
print(a, l)
img_teste2 = redimensiona_img(img_teste, l, a)
plt.imshow(img_teste2)
plt.axis('off')  
plt.show()
img_teste2.shape

In [ ]:
plt.imshow(cv2.resize(img_teste, (28,28)))
plt.axis('off')  
plt.show()

## Normalização

In [ ]:
def normalizacao(img):
  img = img.astype('float32') / 255.0
  img = np.expand_dims(img, axis=-1)
  return img

In [ ]:
img_teste2.shape, normalizacao(img_teste2).shape

## Processamento das detecções

In [ ]:
def processa_caixa(gray, x, y, w, h):
  roi = extra_roi(gray)
  limiar = limiarizacao(roi)
  (a, l) = limiar.shape
  redimensionada = redimensiona_img(limiar, l, a)
  plt.imshow(redimensionada)
  plt.axis('off')  
  plt.show()
  normalizada = normalizacao(redimensionada)
  caracteres.append((normalizada, (x, y, w, h)))

In [ ]:
for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

In [ ]:
caracteres[0]

In [ ]:
caixas = [caixa[1] for caixa in caracteres]
caixas

In [ ]:
caracteres = np.array([c[0] for c in caracteres], dtype='float32')

In [ ]:
caracteres

# Reconhecimento dos caracteres

In [ ]:
numeros = "0123456789"
letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lista_caracteres = numeros + letras
lista_caracteres = [l for l in lista_caracteres]

In [ ]:
print(lista_caracteres)

In [ ]:
caracteres[0].shape

In [ ]:
caracteres.shape

In [ ]:
previsoes = rede_neural.predict(caracteres)

In [ ]:
previsoes

In [ ]:
previsoes.shape

In [ ]:
caixas

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  #print(i)
  probabilidade = previsoes[i]
  #print(probabilidade)
  caractere = lista_caracteres[i]
  #print(caractere)

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  plt.imshow(img_cp)
  plt.axis('off')  
  plt.show()

In [ ]:
def extrai_roi(img, margem=2):
  roi = img[y - margem:y + h + margem, x - margem:x + w + margem]
  return roi

In [ ]:
conts = encontrar_contornos(dilatado.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

In [ ]:
caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")
previsoes = rede_neural.predict(caracteres)

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  probabilidade = previsoes[i]
  caractere = lista_caracteres[i]

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  plt.imshow(img_cp)
  plt.axis('off')  
  plt.show()

# Outros testes

In [ ]:
def preprocessar_img(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  desfoque = cv2.GaussianBlur(gray, (3, 3), 7)
  bordas = cv2.Canny(desfoque, 40, 150)
  dilatado = cv2.dilate(bordas, np.ones((3,3)))
  return gray, dilatado

In [ ]:
def predicao(previsoes, lista_caracteres):
  i = np.argmax(previsoes)
  probabilidade = previsoes[i]
  caractere = lista_caracteres[i]
  return i, probabilidade, caractere

In [ ]:
def desenhar_na_img(img_cp, caractere):
  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)

## O problema do 0 e O, 1 e I, 5 e S

In [ ]:
img = cv2.imread('/content/1.png')
cv2_imshow(img)

In [ ]:
gray, img_processada = preprocessar_img(img)
cv2_imshow(gray)
cv2_imshow(img_processada)

In [ ]:
conts = encontrar_contornos(img_processada.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")
previsoes = rede_neural.predict(caracteres)

In [ ]:
numeros_2 = "OI234S6789"
letras_2 = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lista_caracteres_2 = numeros_2 + letras_2
lista_caracteres_2 = [l for l in lista_caracteres_2]

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i, probabilidade, caractere = predicao(previsoes, lista_caracteres_2)
  desenhar_na_img(img_cp, caractere)
cv2_imshow(img_cp)

## O problema do texto não detectado

In [ ]:
img = cv2.imread('/content/2.png')
cv2_imshow(img)

In [ ]:
gray, img_processada = preprocessar_img(img)
cv2_imshow(gray)
cv2_imshow(img_processada)

In [ ]:
# RETR_EXTERNAL

In [ ]:
conts = encontrar_contornos(img_processada.copy())
img_cp = img.copy()
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  #if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
  roi = gray[y:y + h, x:x + w]
  thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
cv2_imshow(img_cp)

In [ ]:
img.shape

In [ ]:
altura, largura = img.shape[:2]
print(altura, largura)

In [ ]:
contornos_tamanho = sorted(conts, key=cv2.contourArea, reverse=True)

In [ ]:
contornos_tamanho

In [ ]:
for c in contornos_tamanho:
  (x, y, w, h) = cv2.boundingRect(c)

  if (w >= (largura / 2)) and (h >= altura / 2):
    corta = 8
    img_cortada = img[y+corta:y + h - corta, x+corta:x + w - corta]
    cv2_imshow(img_cortada)

In [ ]:
gray, img_processada = preprocessar_img(img_cortada)
cv2_imshow(img_processada)

In [ ]:
conts = encontrar_contornos(img_processada.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")

previsoes = rede_neural.predict(caracteres)

img_cp = img_cortada.copy()

# percorre todas as predições e caixas delimitadoras
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i, probabilidade, caractere = predicao(previsoes, lista_caracteres)
  desenhar_na_img(img_cp, caractere)
cv2_imshow(img_cp)